In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.hodgkinhuxley.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.hodgkinhuxley.HodgkinHuxley import HodgkinHuxley
from lfimodels.hodgkinhuxley.HodgkinHuxleyStatsMoments import HodgkinHuxleyStatsMoments
from delfi.utils.viz import plot_pdf

%matplotlib inline

In [ ]:
true_params, labels_params = utils.obs_params()

seed = None
I, t_on, t_off, dt = utils.syn_current()
m = HodgkinHuxley(I, dt, seed=seed, cython=True)
p = utils.prior(true_params=true_params, seed=seed)
s = HodgkinHuxleyStatsMoments(t_on=t_on, t_off=t_off)
g = dg.Default(model=m, prior=p, summary=s)

obs = utils.syn_obs_data(I, dt, true_params, seed=seed)
obs_stats = utils.syn_obs_stats(I=I, t_on=t_on, t_off=t_off, dt=dt, params=true_params, seed=seed)

In [ ]:
seed = 42
g = dg.Default(model=m, prior=p, summary=s)
res = infer.SNPE(g, 
                 obs=obs_stats, 
                 pilot_samples=1000, 
                 n_hiddens=[50], 
                 seed=seed, 
                 prior_norm=True)

# run with N samples
n_train = 3000
n_rounds = 3
kernel_loss = None
epochs_cbk = None
minibatch_cbk = None
out = res.run(n_train=n_train, n_rounds=n_rounds, 
                  kernel_loss=kernel_loss, epochs_cbk = epochs_cbk, minibatch_cbk=minibatch_cbk)

In [ ]:
prior_min = res.generator.prior.lower
prior_max = res.generator.prior.upper

prior_lims = np.concatenate((prior_min.reshape(-1,1),
                             prior_max.reshape(-1,1)),
                            axis=1)

plot_pdf(out[2][-1].xs[0], lims=prior_lims, samples=None, 
         gt=true_params);

In [ ]:
r = 1
stats, params, w = out[1][r][1], out[1][r][0], out[1][r][2]

bam = np.zeros(w.size)
paramz = (params *res.params_std) + res.params_mean
for i in range(w.size):
    bam[i] = np.all(paramz[i,:] <= res.generator.prior.upper) and np.all(paramz[i,:] >= res.generator.prior.lower)
print('bam', bam.mean())

w /= w.sum()

print('ESS', 1./np.sum(w**2))

idx_worst = np.argmax(w)

plt.figure(figsize=(12,10))
tmp = np.cumsum(np.sort(w))
plt.plot(tmp)
plt.show()

plt.figure(figsize=(12,12))


true_paramz = (true_params - res.params_mean) / res.params_std
plt.plot(params.mean(axis=0).reshape(-1))
plt.plot(true_paramz.reshape(-1) , 'ro-')
plt.plot(params[idx_worst,:], 'g')
plt.show()

plt.figure(figsize=(12,12))

obs_statz = (obs_stats - res.stats_mean) / res.stats_std
plt.plot(stats.mean(axis=0).reshape(-1))
plt.plot(obs_stats.reshape(-1) , 'ro-')
plt.plot(stats[idx_worst,:], 'g')
plt.show()


In [ ]:
seed = 1
g = dg.Default(model=m, prior=p, summary=s)
res_k = infer.SNPE(g, obs=obs_stats, pilot_samples=1000, n_hiddens=[50], seed=seed, prior_norm=True)

# run with N samples
n_train = 3000
n_rounds = 3
kernel_loss = 'x_kl'
epochs_cbk = 10000
minibatch_cbk = n_train
out_k = res_k.run(n_train=n_train, n_rounds=n_rounds, 
                  kernel_loss=kernel_loss, epochs_cbk = epochs_cbk, minibatch_cbk=minibatch_cbk)

In [ ]:
prior_min = res_k.generator.prior.lower
prior_max = res_k.generator.prior.upper

prior_lims = np.concatenate((prior_min.reshape(-1,1),
                             prior_max.reshape(-1,1)),
                            axis=1)

plot_pdf(out_k[2][-1].xs[0], lims=prior_lims, samples=None, 
         gt=true_params);

In [ ]:
r = 4
stats, params, w = out_k[1][r][1], out_k[1][r][0], out_k[1][r][2]
w /= w.sum()

print('ESS', 1./np.sum(w**2))

idx_worst = np.argmax(w)

plt.figure(figsize=(12,10))
tmp = np.cumsum(np.sort(w))
plt.plot(tmp)
plt.show()

plt.plot(out_k[0][r]['cbk_loss'])
plt.show()

print('A', out_k[0][r]['cbkrnl'].A)

In [ ]:
r=1
plt.plot(out_k[0][r]['cbk_loss'])
plt.show()

In [ ]:
for r in range(n_rounds):
    plot_pdf(out[2][r], pdf2=out_k[2][r], lims=prior_lims, samples=None, resolution=100,
         gt=true_params, figsize=(12,12));

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import numpy as np
import os
import scipy.misc
import sys


def run_smc(model, prior, summary, obs_stats, n_params, seed=None,n_particles=1e3,eps_init=2,maxsim=5e7,
            dist_=None, ps=None):
    """Runs Sequential Monte Carlo ABC algorithm.
    Adapted from epsilonfree code https://raw.githubusercontent.com/gpapamak/epsilon_free_inference/8c237acdb2749f3a340919bf40014e0922821b86/demos/mg1_queue_demo/mg1_abc.py

    Parameters
    ----------
    model : 
         Model
    prior :
         Prior
    summary :
         Function to compute summary statistics
    obs_stats: 
         Observed summary statistics
    n_params : 
         Number of parameters
    seed : int or None
        If set, randomness in sampling is disabled
    n_particles : int
        Number of particles for SMC-ABC
    eps_init : Float
        Initial tolerance for SMC-ABC
    maxsim : int
        Maximum number of simulations for SMC-ABC
    """
    n_particles = int(n_particles)
    
    if dist_ is None:
        dist_ = calc_dist
    # check for subfolders, create if they don't exist
    dirs = {}
    dirs['dir_abc'] = './results/abc/'
    for k, v in dirs.items():
        if not os.path.exists(v):
            os.makedirs(v)

    prefix = str(n_params)+'params'
    
    #####################
    np.random.seed(seed)

    # set parameters
    eps_last = 0.01
    eps_decay = 0.9
    ess_min = 0.5
    maxsim = int(maxsim)

    all_ps = []
    all_logweights = []
    all_eps = []
    all_nsims = []

    # sample initial population
    weights = np.ones(n_particles, dtype=float) / n_particles
    logweights = np.log(weights)
    eps = eps_init
    iter = 0
    nsims = 0

    if ps is None:
        ps = np.empty([n_particles, n_params])

        for i in range(n_particles):

            dist = float('inf')

            while dist > eps:
                ps[i] = prior.gen(n_samples=1)[0]
                states = model.gen_single(ps[i])
                stats = summary.calc([states])
                dist = dist_(stats, obs_stats)
                nsims += 1
                
    else: 
        assert ps.shape == (n_particles, n_params)
        assert ps.dtype == float

    all_ps.append(ps)
    all_logweights.append(logweights)
    all_eps.append(eps)
    all_nsims.append(nsims)

    break_flag = False

    print('iteration = {0}, eps = {1:.2}, ess = {2:.2%}'.format(iter, float(eps), 1.0))

    while eps > eps_last:

        iter += 1
        eps *= eps_decay

        # calculate population covariance
        mean = np.mean(ps, axis=0)
        cov = 2.0 * (np.dot(ps.T, ps) / n_particles - np.outer(mean, mean))
        std = np.linalg.cholesky(cov)

        # perturb particles
        new_ps = np.empty_like(ps)
        new_logweights = np.empty_like(logweights)

        for i in range(n_particles):

            dist = float('inf')

            while dist > eps:
                idx = discrete_sample(weights)[0]
                new_ps[i] = ps[idx] + np.dot(std, np.random.randn(n_params))
                states = model.gen_single(new_ps[i])
                stats = summary.calc([states])
                dist = dist_(stats, obs_stats)
                nsims += 1
                if nsims>=maxsim:
                    print('Maximum number of simulations reached.')
                    break_flag = True
                    break
            #new_ps_i = new_ps[i]
            logkernel = -0.5 * np.sum(np.linalg.solve(std, (new_ps[i] - ps).T) ** 2, axis=0)
            #new_logweights[i] = -float('inf') if np.any(new_ps_i > prior_max) or np.any(new_ps_i < prior_min) else -scipy.misc.logsumexp(logweights + logkernel)
            new_logweights[i] = prior.eval(new_ps[i, np.newaxis], log=True)[0] - scipy.misc.logsumexp(logweights + logkernel)

            if break_flag:
                break

        if break_flag:
            break

        ps = new_ps
        logweights = new_logweights - scipy.misc.logsumexp(new_logweights)
        weights = np.exp(logweights)

        # calculate effective sample size
        ess = 1.0 / (np.sum(weights ** 2) * n_particles)
        print('iteration = {0}, eps = {1:.2}, ess = {2:.2%}'.format(iter, float(eps), ess))

        if ess < ess_min:

            # resample particles
            new_ps = np.empty_like(ps)

            for i in range(n_particles):
                idx = discrete_sample(weights)[0]
                new_ps[i] = ps[idx]

            ps = new_ps
            weights = np.ones(n_particles, dtype=float) / n_particles
            logweights = np.log(weights)

        all_ps.append(ps)
        all_logweights.append(logweights)
        all_eps.append(eps)
        all_nsims.append(nsims)

    return all_ps, all_logweights, all_eps, all_nsims


        
def calc_dist(stats_1, stats_2):
    """Euclidian distance between summary statistics"""
    return np.sqrt(np.sum((stats_1 - stats_2) ** 2))

def discrete_sample(p, n_samples=1):
    """
    Samples from a discrete distribution.
    :param p: a distribution with N elements
    :param n_samples: number of samples
    :return: vector of samples
    """

    # check distribution
    #assert isdistribution(p), 'Probabilities must be non-negative and sum to one.'

    # cumulative distribution
    c = np.cumsum(p[:-1])[np.newaxis, :]

    # get the samples
    r = np.random.rand(n_samples, 1)
    return np.sum((r > c).astype(int), axis=1)

In [ ]:
import sys
sys.path.append('../nips_2017/')
#from scripts.run_abc import run_smc as run_smc 
import delfi.utils.io as io
import time

def z_dist(stats_1, stats_2):
    """Euclidian distance between summary statistics"""
    statz_1 = (stats_1 - res.stats_mean) / res.stats_std
    statz_2 = (stats_2 - res.stats_mean) / res.stats_std
    return np.sqrt(np.sum((statz_1 - statz_2) ** 2))


n_particles=1e3
eps_init=0.8
maxsim=1e4
seed=42

ps = out[2][-1].gen(int(n_particles))

stats = np.vstack([s.calc(datum) for datum in m.gen(ps)])
statz = (stats - res.stats_mean) / res.stats_std
obs_statz = (obs_stats - res.stats_mean) / res.stats_std
dists = np.hstack([z_dist(obs_stats, stats[i,:]) for i in range(statz.shape[0])])
plt.hist(dists)
plt.show()

for i in range(20):
    plt.plot(np.sort(statz[:,i]))
    plt.plot(np.arange(statz.shape[0]), np.ones(statz.shape[0])*obs_statz[0,i], 'r')
    plt.show()

In [ ]:
import sys
sys.path.append('../nips_2017/')
#from scripts.run_abc import run_smc as run_smc 
import delfi.utils.io as io
import time

def z_dist(stats_1, stats_2):
    """Euclidian distance between summary statistics"""
    statz_1 = (stats_1 - res.stats_mean) / res.stats_std
    statz_2 = (stats_2 - res.stats_mean) / res.stats_std
    return np.sqrt(np.sum((statz_1 - statz_2) ** 2))


n_particles=1e3
eps_init=0.8
maxsim=1e4
seed=42

ps = p.gen(int(n_particles))

stats = np.vstack([s.calc(datum) for datum in m.gen(ps)])
statz = (stats - res.stats_mean) / res.stats_std
obs_statz = (obs_stats - res.stats_mean) / res.stats_std
dists = np.hstack([z_dist(obs_stats, stats[i,:]) for i in range(statz.shape[0])])
plt.hist(dists)
plt.show()

for i in range(20):
    plt.plot(np.sort(statz[:,i]))
    plt.plot(np.arange(statz.shape[0]), np.ones(statz.shape[0])*obs_statz[0,i], 'r')
    plt.show()

In [ ]:

n_params = p.ndim
t = time.time()
ps_smc, logweights_smc, eps_smc, all_nsims_smc = run_smc(model=m, prior=p, summary=s,
                                                         ps=ps,
                                                         dist_=z_dist,
                                                         obs_stats=obs_stats,
                                                         n_params=n_params, 
                                                         seed=seed, 
                                                         n_particles=n_particles,
                                                         eps_init=eps_init,
                                                         maxsim=maxsim)

weights_smc = np.exp(logweights_smc)
nsims_smc = np.asarray(all_nsims_smc)

m_smc = []
cov_smc = []
for i in range(len(ps_smc)):
    m_smc.append(np.dot(weights_smc[i],ps_smc[i]))
    cov_smc.append(np.cov(ps_smc[i].T,aweights = weights_smc[i]))
    
print('total time', time.time() -t)

p_smc = dd.Gaussian(m=m_smc[-1], S=cov_smc[-1])
sam = p_smc.gen(1000) #(p_smc.gen(1000) - res_k.params_mean) / res_k.params_std

In [ ]:
np.diag(out[2][-1].xs[0].S)

In [ ]:
np.diag(cov_smc[-1])

In [ ]:
ps_smc[-1].shape, sam.shape

In [ ]:
sam = ps_smc[-1]
for r in range(n_rounds-1, n_rounds):
    plot_pdf(out[2][r], pdf2=out_k[2][r], lims=prior_lims, samples=sam.T, resolution=100,
         gt=true_params, figsize=(12,12));

In [ ]:
import sys
sys.path.append('../nips_2017/')
#from scripts.run_abc import run_smc as run_smc 
import delfi.utils.io as io
import time

n_particles=1e3
eps_init=0.1
maxsim=2e4
seed=42
ps = out_k[2][-1].gen(int(n_particles))

def z_dist(stats_1, stats_2):
    """Euclidian distance between summary statistics"""
    statz_1 = (stats_1 - res.stats_mean) / res.stats_std
    statz_2 = (stats_2 - res.stats_mean) / res.stats_std
    return np.sqrt(np.sum((statz_1 - statz_2) ** 2))

#stats = np.vstack([s.calc(datum) for datum in m.gen(ps)])
#statz = (stats - res.stats_mean) / res.stats_std
#obs_statz = (obs_stats - res.stats_mean) / res.stats_std
#dists = np.hstack([z_dist(obs_stats, stats[i,:]) for i in range(statz.shape[0])])
#plt.hist(dists)
#plt.show()

#for i in range(20):
#    plt.plot(np.sort(statz[:,i]))
#    plt.plot(np.arange(statz.shape[0]), np.ones(statz.shape[0])*obs_statz[0,i], 'r')
#    plt.show()

n_params = p.ndim
t = time.time()
ps_smc, logweights_smc, eps_smc, all_nsims_smc = run_smc(model=m, prior=p, summary=s,
                                                         ps=ps,
                                                         dist_=z_dist,
                                                         obs_stats=obs_stats,
                                                         n_params=n_params, 
                                                         seed=seed, 
                                                         n_particles=n_particles,
                                                         eps_init=eps_init,
                                                         maxsim=maxsim)

weights_smc = np.exp(logweights_smc)
nsims_smc = np.asarray(all_nsims_smc)

m_smc = []
cov_smc = []
for i in range(len(ps_smc)):
    m_smc.append(np.dot(weights_smc[i],ps_smc[i]))
    cov_smc.append(np.cov(ps_smc[i].T,aweights = weights_smc[i]))
    
print('total time', time.time() -t)

p_smc = dd.Gaussian(m=m_smc[-1], S=cov_smc[-1])
sam = p_smc.gen(1000) #(p_smc.gen(1000) - res_k.params_mean) / res_k.params_std